In [6]:
import cv2
import sys
import os
import time
import dlib
import random
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from sklearn.cross_validation  import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [11]:
#detect face by haarcascade method 
cascPath = "haarcascade_frontalface_default.xml"
dataPath='D:\\code\\pycode\\faceRecognition\\database\\'
patchPath='D:\\code\\pycode\\faceRecognition\\faceData\\'
faceCascade = cv2.CascadeClassifier(cascPath)
for personFolder in os.listdir(dataPath):
    personPath=os.path.join(dataPath,personFolder)
    for fileName in os.listdir(personPath):
        image = cv2.imread(os.path.join(personPath,fileName))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30, 30))
        if(len(faces)==1):
            x,y,w,h=list(faces[0])
            facePatch=image[y:y+h,x:x+w]
            if(not os.path.exists(os.path.join(patchPath,personFolder))):
                os.mkdir(os.path.join(patchPath,personFolder)) 
            cv2.imwrite(os.path.join(patchPath+personFolder,fileName),facePatch)

In [ ]:
detector = dlib.get_frontal_face_detector()
dataPath='D:\\code\\pycode\\faceRecognition\\database\\'
patchPath='D:\\code\\pycode\\faceRecognition\\faceData1\\'
detector = dlib.get_frontal_face_detector()
datagen = ImageDataGenerator(
        rotation_range=20,
#        width_shift_range=0.2,
#        height_shift_range=0.2,
        rescale=None,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        data_format='channels_last',
        cval=0,
        #channel_shift_range=0,
        vertical_flip=False)
for personFolder in os.listdir(dataPath):
    personPath=os.path.join(dataPath,personFolder)
    for fileName in os.listdir(personPath):
        image = cv2.imread(os.path.join(personPath,fileName))
        dets = detector(image, 1)
        for i, d in enumerate(dets):
            print("person{} face{} Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(personFolder,fileName[-5:-4],i, d.left(), d.top(), d.right(), d.bottom()))
            facePatch=image[d.top():d.bottom(),d.left():d.right()]
            cv2.imwrite(os.path.join(patchPath+personFolder,fileName),facePatch)
            
            facePatch=img_to_array(facePatch)
            facePatch = facePatch.reshape((1,) + facePatch.shape)
            if(not os.path.exists(os.path.join(patchPath,personFolder))):
                os.mkdir(os.path.join(patchPath,personFolder)) 
                
            i = 0
            for batch in datagen.flow(facePatch, batch_size=5,
                                      save_to_dir=None#os.path.join(patchPath,personFolder), 
                                      #save_prefix=personFolder, 
                                      #save_format='jpeg'
                                      ):
                cv2.imwrite(os.path.join(patchPath+personFolder,fileName[:-4]+str(i)+'.JPG'),batch[0].astype(int))
                i += 1
                if i > 5:
                    break  # 否则生成器会退出循环
            


In [9]:
trainData=[[]]
label=[]
dataPath='D:\\code\\pycode\\faceRecognition\\faceData1\\'
for dataLabel in os.listdir(dataPath):
    folderPath=os.path.join(dataPath,dataLabel)
    for fileName in os.listdir(folderPath):
        image=cv2.imread(os.path.join(folderPath,fileName))
        gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        imageArray=np.array(cv2.resize(gray_image,(101,101))).reshape([101*101])/255
        label.append(int(dataLabel))
        if(trainData==[[]]):
            trainData = [imageArray]
            continue
        trainData = np.concatenate((trainData, [imageArray]),axis=0)
pca = PCA(n_components=0.95)
pca.fit(trainData)
print ("explained_variance_ratio_",pca.explained_variance_ratio_)
print ("explained_variance_",pca.explained_variance_)
print ("pca.n_components_",pca.n_components_)
label=np.array(label)
trainData=pca.transform(trainData)
train_images, test_images, train_labels, test_labels = train_test_split(trainData, label, test_size = 0.05, random_state = random.randint(0, 100)) 

C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.


explained_variance_ratio_ [3.93998513e-01 1.09799949e-01 6.00968807e-02 3.55681508e-02
 3.31248263e-02 2.71910742e-02 2.59669404e-02 1.70909728e-02
 1.54459024e-02 1.24931933e-02 1.15359351e-02 9.26532549e-03
 8.76926262e-03 7.96429647e-03 7.27845752e-03 7.05500326e-03
 5.92457055e-03 5.33406407e-03 4.84517223e-03 4.54949433e-03
 4.31949953e-03 4.16861368e-03 3.90338113e-03 3.78951346e-03
 3.56169932e-03 3.37028291e-03 3.31888319e-03 3.11639599e-03
 3.00885387e-03 2.79066883e-03 2.75592404e-03 2.54578244e-03
 2.45663604e-03 2.38639067e-03 2.26100351e-03 2.18234805e-03
 2.05084226e-03 1.99210183e-03 1.92708066e-03 1.87489906e-03
 1.75570632e-03 1.69616748e-03 1.67336380e-03 1.64122965e-03
 1.58641294e-03 1.54524640e-03 1.50113685e-03 1.46904912e-03
 1.41827721e-03 1.40840145e-03 1.31154400e-03 1.30192037e-03
 1.28305611e-03 1.24835514e-03 1.22617390e-03 1.21820976e-03
 1.17818817e-03 1.14511577e-03 1.12322674e-03 1.10523512e-03
 1.05720031e-03 1.01818605e-03 1.00364047e-03 9.90450568e-0

In [ ]:
svc = SVC(gamma='auto')
svc.fit(train_images,train_labels) 
test_pred=svc.predict(test_images)   #用测试数据来做预测
print(classification_report(test_labels, test_pred))

In [ ]:
clf = RandomForestClassifier(n_jobs=111)
clf.fit(train_images,train_labels) 
test_pred=clf.predict(test_images)  #用测试数据来做预测
print(classification_report(test_labels,test_pred))

In [ ]:
clf = AdaBoostClassifier(n_estimators=100)


In [9]:
imagePath='D:\\code\\pycode\\faceRecognition\\database\\99\\MVC-007F.JPG'
image=cv2.imread(imagePath)
gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.resize(gray_image,(101,101))
gray_image.shape

(480, 640)

In [ ]:
images=io.imread('D:\\code\\pycode\\faceRecognition\\database\\99\\MVC-007F.JPG')

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'，
        cval=0,
        channel_shift_range=0,
        horizontal_flip=False,
        vertical_flip=False,
        rescale=None)
#plt.imshow(images_aug2)
#plt.imshow(images_aug3)
#plt.imshow(images_aug4)